In [1]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "cv_generator"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

In [2]:
import os
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import datetime
from langchain_community.callbacks import get_openai_callback

# Récupérer la clé API OpenAI depuis les variables d'environnement
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("La clé API OpenAI n'est pas définie dans les variables d'environnement.")

# Modèle local Mistral via Ollama
llm = ChatOllama(model="mistral", temperature=0, base_url="http://127.0.0.1:11500")

# Modèle GPT-4o via OpenAI
llm2 = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key  # Utilisation de la clé API OpenAI depuis l'env
)

# Test des modèles
#print("Réponse Mistral:", llm.invoke("Hello, world!"))  # Mistral
print("Réponse GPT-4o mini:", llm2.invoke("Hello, world!"))  # GPT-4o

/var/folders/kz/q2b2x49j16x2jfrqxg8_grdc0000gn/T/ipykernel_11755/2630718512.py:15: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="mistral", temperature=0, base_url="http://127.0.0.1:11500")


Réponse GPT-4o mini: content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-2721a673-503f-4113-bd5b-89b436fc886f-0' usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# Import du texte source

On importe le texte et on en extrait les principales expériences de façon destructuré

In [3]:
import json
from pathlib import Path
# Charger le contenu du fichier source
source_file = Path("source_brut.txt")
if not source_file.exists():
    raise FileNotFoundError("Le fichier source_brut.txt est introuvable.")

with source_file.open("r", encoding="utf-8") as f:
    source_text = f.read()

time_start = datetime.datetime.now()
total_tokens = 0
prompt_tokens = 0
completion_tokens = 0
total_cost = 0

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Étape 1 : Génération de la liste des expériences et formations
summary_prompt = ChatPromptTemplate.from_template(
    """
    À partir du texte suivant qui décrit un profil candidat :
    """
    "{source}"
    """
    Extrait une liste concise de ses expériences et formations sous la forme suivante :
    - Expérience : Intitulé, Dates, Lieu
    - Éducation : Intitulé, Dates, Lieu
    """
)
summary_chain = summary_prompt | llm2 | StrOutputParser()

# Exécution de la chaîne en capturant le coût via le callback
with get_openai_callback() as cb:
    summary_text = summary_chain.invoke({"source": source_text})


In [ ]:
print(summary_text)

# Transormation en JSON

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from typing import List

# Définir la structure de données souhaitée

class Experience(BaseModel):
    intitule: str = Field(description="Intitulé du poste ou de l'expérience")
    dates: str = Field(description="Période de l'expérience")
    etablissement: str = Field(description="Nom de l'établissement")
    lieu: str = Field(description="Lieu de l'expérience")

class Education(BaseModel):
    intitule: str = Field(description="Intitulé du diplôme ou de la formation")
    dates: str = Field(description="Période de la formation")
    etablissement: str = Field(description="Nom de l'établissement")
    lieu: str = Field(description="Lieu de l'établissement")

class CV(BaseModel):
    experiences: List[Experience] = Field(description="Liste des expériences professionnelles")
    education: List[Education] = Field(description="Liste des formations")

# Instancier le parser JSON avec le modèle CV
parser = JsonOutputParser(pydantic_object=CV)

# Créer un prompt en injectant les instructions de format fournies par le parser
prompt = PromptTemplate(
    template="""
Transforme le texte suivant en un JSON structuré avec deux clés : "experiences" et "education".
Le format attendu est :
{format_instructions}

Texte :
"{summary}"
""",
    input_variables=["summary"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Construire la chaîne en combinant prompt, modèle de langage et parser
chain = prompt | llm2 | parser

# Exécution de la chaîne en capturant le coût via le callback
with get_openai_callback() as cb1:
    json_output = chain.invoke({"summary": summary_text})

print(json_output)

# Enrichissement des exp

In [ ]:
# Conversion du JSON en dictionnaire
parsed_json = json_output

# Étape 3 : Ajout d'une description à chaque expérience et formation
description_prompt = ChatPromptTemplate.from_template(
    """
    Voici le profil détaillé du candidat :
    "{source}"
    
    Basé sur ces informations, rédige une description extrêmement détaillée et exhaustive de l'expérience suivante, en incluant **toutes** les informations disponibles dans le texte source :
    
    - Intitulé : {intitule}
    - Dates : {dates}
    - Établissement : {etablissement}
    - Lieu : {lieu}
    
    Intègre les responsabilités, les missions précises, les compétences mises en œuvre, les résultats obtenus, les outils et technologies utilisées, les collaborations, les projets majeurs associés, les distinctions éventuelles et toute autre information pertinente.
    
    Donne uniquement la description enrichie en sortie, sans aucun autre texte additionnel.
    Ne rappelle pas les informations sur les dates, l'établissement ou le lieu.
    """
)


def enrich_with_description(entry):
    global total_tokens, prompt_tokens, completion_tokens, total_cost
    description_chain = description_prompt | llm2 | StrOutputParser()

    # Exécution de la chaîne en capturant le coût via le callback
    with get_openai_callback() as cb2:
        description = description_chain.invoke({"source": source_text, **entry})
    
    total_tokens += cb.total_tokens
    prompt_tokens += cb.prompt_tokens
    completion_tokens += cb.completion_tokens
    total_cost += cb.total_cost

    entry["description"] = description
    return entry

parsed_json["experiences"] = [enrich_with_description(exp) for exp in parsed_json["experiences"]]
parsed_json["education"] = [enrich_with_description(edu) for edu in parsed_json["education"]]



In [ ]:
# Calcul du temps d'exécution
time_diff = datetime.datetime.now() - time_start
execution_time_sec = time_diff.total_seconds()

# Ajout des informations de coût dans le JSON
parsed_json["total_tokens"] = cb.total_tokens + cb1.total_tokens + total_tokens
parsed_json["prompt_tokens"] = cb.prompt_tokens + cb1.prompt_tokens + prompt_tokens
parsed_json["completion_tokens"] = cb.completion_tokens + cb1.completion_tokens + completion_tokens
parsed_json["total_cost"] = cb.total_cost + cb1.total_cost + total_cost


#on ajoute les clés au json
parsed_json["source_file"] = source_file.name
parsed_json["date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
parsed_json["method"] = "modulaire"
parsed_json["model"] = "gpt-4o-mini"
parsed_json["execution_time"] = execution_time_sec


# Sauvegarde du résultat dans un fichier JSON avec le timestamp
output_file = Path(f"output_mod_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.json")
with output_file.open("w", encoding="utf-8") as f:
    json.dump(parsed_json, f, indent=2, ensure_ascii=False)

print("JSON enrichi avec descriptions et sauvegardé dans output.json")
